In [41]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
edad_seccion_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\07 - Edad\Secciones\Ocupados de 16 y más años_Edad_Secciones_Censo_2011.csv"
edad_seccion_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\07 - Edad\Secciones\Ocupados de 16 y más años_Edad_Secciones_Censo_2021.csv"
cartografia_seccion_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaSeccionCensal-25830\da_cartografiaSeccionCensalPolygon.shp"

## Lectura inicial de datos lugar de nacimiento
df_edad_seccion_2011 =  pd.read_csv(edad_seccion_2011_csv_path,sep=";")
df_edad_seccion_2021 =  pd.read_csv(edad_seccion_2021_csv_path,sep=";")
df_cartografia_seccion = gpd.read_file(cartografia_seccion_shp_path)

# Ver las columnas
#print(df_edad_seccion_2011.columns)
#print(df_edad_seccion_2021.columns)

# Inspeccionar los primeros registros
#print(df_edad_seccion_2011.head())
#print(df_edad_seccion_2021.head())

#Cambio el nombre de la columna 'Distrito ' a Distrito de residencia
df_edad_seccion_2021.rename(columns={'Sección ': 'Seccion de residencia'}, inplace=True)

#Se realiza una normalizacion de los csv  
df_edad_seccion_2011_long = pd.melt(df_edad_seccion_2011, id_vars=["Municipio de residencia","Seccion de residencia"], var_name='Edad', value_name='Personas')
#print(df_edad_seccion_2011_long.columns)
#print(df_edad_seccion_2011_long.head())
df_edad_seccion_2021_long = pd.melt(df_edad_seccion_2021, id_vars=["Municipio de residencia","Seccion de residencia"], var_name='Edad', value_name='Personas')
#print(df_edad_seccion_2021_long.columns)
#print(df_edad_seccion_2021_long.head())

## Eliminar filas ##
#Eliminar filas de municipio Total
df_edad_seccion_2011_long = df_edad_seccion_2011_long[df_edad_seccion_2011_long['Municipio de residencia'] != 'Total']
df_edad_seccion_2021_long = df_edad_seccion_2021_long[df_edad_seccion_2021_long['Municipio de residencia'] != 'Total']
#Eliminar filas de Seccion de residencia Total
df_edad_seccion_2011_long = df_edad_seccion_2011_long[df_edad_seccion_2011_long['Seccion de residencia'] != 'Total']
df_edad_seccion_2021_long = df_edad_seccion_2021_long[df_edad_seccion_2021_long['Seccion de residencia'] != 'Total']
#Eliminar filas de Edad Total
df_edad_seccion_2011_long = df_edad_seccion_2011_long[df_edad_seccion_2011_long['Edad'] != 'Total']
df_edad_seccion_2021_long = df_edad_seccion_2021_long[df_edad_seccion_2021_long['Edad'] != 'Total']

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_edad_seccion_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_edad_seccion_2011_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_edad_seccion_2011_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#6401
#print(num_filas_contiene_asterisco_2011)#6401
num_filas_en_blanco_2021 = (df_edad_seccion_2021_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_edad_seccion_2021_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_edad_seccion_2021_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

## Añadir columnas ##
#Añadir columna NUMSECCENS con el NUM de la seccion
df_edad_seccion_2011_long["NUMSECCENS"] = df_edad_seccion_2011_long["Seccion de residencia"].str[-5:].astype(float)
df_edad_seccion_2021_long["NUMSECCENS"] = df_edad_seccion_2021_long["Seccion de residencia"].str[-5:].astype(float)
print(df_edad_seccion_2011_long.head())
print(df_edad_seccion_2021_long.head())

## Gestion de Nulos y valores extraños ##
# Limpiar los datos: eliminar el asterisco al final, pero dejar intacto el '*'
df_edad_seccion_2011_long['Personas'] = df_edad_seccion_2011_long['Personas'].replace(r'(\d+)\*$', r'\1', regex=True)
df_edad_seccion_2021_long['Personas'] = df_edad_seccion_2021_long['Personas'].replace(r'(\d+)\*$', r'\1', regex=True)
#Sustituimos los valores * por NA
df_edad_seccion_2011_long['Personas'] = df_edad_seccion_2011_long['Personas'].replace('*', np.nan)
df_edad_seccion_2021_long['Personas'] = df_edad_seccion_2021_long['Personas'].replace('*', np.nan)
#Columna Personas como float
df_edad_seccion_2011_long['Personas'] = df_edad_seccion_2011_long['Personas'].astype(float)
df_edad_seccion_2021_long['Personas'] = df_edad_seccion_2021_long['Personas'].astype(float)

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_edad_seccion_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_edad_seccion_2011_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_edad_seccion_2011_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#6401
#print(num_filas_con_asterisco_2011)#0
#print(num_filas_contiene_asterisco_2011)#0
num_filas_en_blanco_2021 = (df_edad_seccion_2021_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_edad_seccion_2021_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_edad_seccion_2021_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

#Hay 6401 valores vacios que sabemos eran * y segun el INE su valor esta entre 1-5. Usaremos el 3 como valor arbitrario (media)
# Calcular la desviación estándar antes de la imputación
desviacion_imputada_2011= df_edad_seccion_2011_long.groupby('Seccion de residencia')['Personas'].std()
#print("Desviación estándar por seccion antes de imputar 3:", desviacion_imputada_2011)
#Vamos a imputar un 3 y ver si hay mucha desviacion
df_edad_seccion_2011_long['Personas'] = df_edad_seccion_2011_long['Personas'].replace(np.nan, 3)
df_edad_seccion_2021_long['Personas'] = df_edad_seccion_2021_long['Personas'].replace(np.nan, 3)
# Calcular la desviación estándar despues de la imputación
desviacion_imputada_2011= df_edad_seccion_2011_long.groupby('Seccion de residencia')['Personas'].std()
#print("Desviación estándar por seccion despues de imputar 3:", desviacion_imputada_2011)

#Vamos a reducir el numero de categorias a  15-19 , 20-34, 35-64 y 65+
# Crear la categoria 20-34
def group_20_34(df):
    # Filtrar por los registros de divorciados y separados
    filtered = df[df["Edad"].isin(["20-24", "25-29", "30-34"])]
    
    # Agrupar y sumar las personas por Municipio y Distrito
    grouped = filtered.groupby(["Municipio de residencia", "Seccion de residencia","NUMSECCENS"], as_index=False)["Personas"].sum()
    grouped["Edad"] = "20-34"

    return grouped

# Crear la categoria 35-64
def group_35_64(df):
    # Filtrar por los registros de divorciados y separados
    filtered = df[df["Edad"].isin(["35-39", "40-44", "45-49", "50-54", "55-59", "60-64"])]
    
    # Agrupar y sumar las personas por Municipio y Distrito
    grouped = filtered.groupby(["Municipio de residencia", "Seccion de residencia","NUMSECCENS"], as_index=False)["Personas"].sum()
    grouped["Edad"] = "35-64"

    return grouped

# Crear la categoria 65+
def group_65(df):
    # Filtrar por los registros de divorciados y separados
    filtered = df[df["Edad"].isin(["65-69", "70-74", "75-79", "80-84", "85-89", "90-94", "95-99"])]
    
    # Agrupar y sumar las personas por Municipio y Distrito
    grouped = filtered.groupby(["Municipio de residencia", "Seccion de residencia","NUMSECCENS"], as_index=False)["Personas"].sum()
    grouped["Edad"] = "65+"

    return grouped

# Obtener los datos agrupados
grouped_data_20_34_2011 = group_20_34(df_edad_seccion_2011_long)
grouped_data_20_34_2021 = group_20_34(df_edad_seccion_2021_long)
grouped_data_35_64_2011 = group_35_64(df_edad_seccion_2011_long)
grouped_data_35_64_2021 = group_35_64(df_edad_seccion_2021_long)
grouped_data_65_2011 = group_65(df_edad_seccion_2011_long)
grouped_data_65_2021 = group_65(df_edad_seccion_2021_long)

# Eliminar las filas originales y añadir la nueva categoría
df_edad_seccion_2011_long = pd.concat([
    df_edad_seccion_2011_long[df_edad_seccion_2011_long["Edad"].isin(["15-19"])],
    grouped_data_20_34_2011,
    grouped_data_35_64_2011,
    grouped_data_65_2011
], ignore_index=True)

df_edad_seccion_2021_long = pd.concat([
    df_edad_seccion_2021_long[df_edad_seccion_2021_long["Edad"].isin(["15-19"])],
    grouped_data_20_34_2021,
    grouped_data_35_64_2021,
    grouped_data_65_2021
], ignore_index=True)

#print(df_edad_seccion_2011_long.count())#1712 registros
#print(df_edad_seccion_2021_long.count())#544 registros
#print(df_edad_seccion_2011_long.head())
#print(df_edad_seccion_2021_long.head())

##Calcular el total de personas por distrito y el porcentaje de Personas
df_edad_seccion_2011_long['Total_personas_seccion'] = df_edad_seccion_2011_long.groupby('Seccion de residencia')['Personas'].transform('sum')
df_edad_seccion_2021_long['Total_personas_seccion'] = df_edad_seccion_2021_long.groupby('Seccion de residencia')['Personas'].transform('sum')
#Calcular porcentaje sobre el total
df_edad_seccion_2011_long['Personas %'] = (df_edad_seccion_2011_long['Personas'] / df_edad_seccion_2011_long['Total_personas_seccion'])
df_edad_seccion_2021_long['Personas %'] = (df_edad_seccion_2021_long['Personas'] / df_edad_seccion_2021_long['Total_personas_seccion'])
#print(df_edad_seccion_2011_long[df_edad_seccion_2011_long['NUMSECCENS'] == 1001])
#print(df_edad_seccion_2011_long[df_edad_seccion_2011_long['NUMSECCENS'] == 3])
#print(df_edad_seccion_2011_long[df_edad_seccion_2011_long['NUMSECCENS'] == 5])
#print(df_edad_seccion_2021_long[df_edad_seccion_2021_long['NUMSECCENS'] == 1001])

#Pivotar la tabla de nuevo para crear 8 variables, una por categoria de Edad con el % ; una fila por seccion
df_edad_seccion_2011_long  = df_edad_seccion_2011_long.pivot(index='NUMSECCENS', columns='Edad', values='Personas %')
df_edad_seccion_2021_long  = df_edad_seccion_2021_long.pivot(index='NUMSECCENS', columns='Edad', values='Personas %')

print(df_edad_seccion_2011_long)
print(df_edad_seccion_2011_long.count())#428
print(df_edad_seccion_2021_long)
print(df_edad_seccion_2021_long.count())#136

  Municipio de residencia Seccion de residencia   Edad Personas  NUMSECCENS
0            29067 Málaga    Sección 2906701001  15-19        *      1001.0
1            29067 Málaga    Sección 2906701002  15-19        *      1002.0
2            29067 Málaga    Sección 2906701003  15-19        *      1003.0
3            29067 Málaga    Sección 2906701004  15-19        *      1004.0
4            29067 Málaga    Sección 2906702001  15-19        *      2001.0
    Municipio de residencia Seccion de residencia   Edad  Personas  NUMSECCENS
412            29067 Málaga    Sección 2906701001  15-19         0      1001.0
413            29067 Málaga    Sección 2906702003  15-19         0      2003.0
414            29067 Málaga    Sección 2906702004  15-19         0      2004.0
415            29067 Málaga    Sección 2906702009  15-19         0      2009.0
416            29067 Málaga    Sección 2906702013  15-19         3      2013.0
Edad           15-19     20-34     35-64       65+
NUMSECCENS         

In [43]:
##Export 4 variables
df_edad_seccion_2011_long.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\07 - Edad\Secciones\2011_Secciones_Edad.csv",index=True,sep=";")
df_edad_seccion_2021_long.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\07 - Edad\Secciones\2021_Secciones_Edad.csv",index=True,sep=";")